# Spark Streaming and Senzing

This notebook shows you how to process streaming data through Apache Spark and send it to Senzing for entity resolution, simulating a real-time data processing pipeline. If you haven't already gone through the `senzing_quickstart` tutorial in this repository, we recommend starting with that one because it contains more detailed explanations for each of the steps.

### Steps in this tutorial

1. Set up the Senzing gRPC server, download the `customer.json` data file and split it into 20 separate JSON files to simulate streaming data.
2. Configure the Senzing engine so it's ready to receive data.
3. Create a Spark session with streaming capabilities, define a schema and set up a streaming dataframe.
4. Implement a batch processing function that takes each streaming batch from Spark, sends individual records to Senzing for entity resolution, and tracks which entities are affected by each record addition.
5. Run a cleanup process to ensure the entities are as accurate as possible.


## Set up requirements

In this tutorial, we'll use the [`senzing`](https://garage.senzing.com/sz-sdk-python/index.html) and [`senzing_grpc`](https://garage.senzing.com/sz-sdk-python-grpc/) packages, in addition to PySpark. You can install all of these using the `requirements.txt` file in the repo folder containing this tutorial.

In [ ]:
import grpc
from senzing import szengineflags, szerror
from senzing_grpc import SzAbstractFactoryGrpc
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import json
import os
import requests
import shutil

We'll start our [Senzing gRPC server](https://github.com/senzing-garage/serve-grpc/tree/main) using Docker.

Run the following command `docker run -it --publish 8261:8261 --rm senzing/serve-grpc` in a terminal window.

Then, we'll download the example data:

In [ ]:

data_path = "./data/"
data_url_prefix = "https://raw.githubusercontent.com/Senzing/truth-sets/refs/heads/main/truthsets/demo/"
filename = "reference.json"

In [ ]:
os.makedirs(data_path, exist_ok=True)

url = data_url_prefix + filename
filepath = data_path + filename
if not os.path.exists(filepath):
    response = requests.get(url, stream=True, timeout=10)
    response.raw.decode_content = True
    with open(filepath, "wb") as file:
        shutil.copyfileobj(response.raw, file)

## Create separate json files to simulate streaming

We'll use the `reference.json` file from the Senzing "Truthsets" demo data located at https://github.com/Senzing/truth-sets/. This file contains customer and organization information, with incomplete contact data.

We'll save each record from the `reference.json` file into a separate json file to simulate streaming data.

In [ ]:
def create_streaming_files(input_file, output_dir):
    
    os.makedirs(output_dir, exist_ok=True)
    
    with open(input_file, 'r') as f:
        for i, line in enumerate(f):

            try:
                record = json.loads(line)
                filename = f"{output_dir}/record_{record['RECORD_ID']}.json"
                
                with open(filename, 'w') as out_file:
                    json.dump(record, out_file)
                    
                print(f"Created {filename}")
                
            except json.JSONDecodeError as e:
                print(f"Error parsing line {i}: {e}")
                continue

In [ ]:
create_streaming_files('data/reference.json', 'data/streaming')

## Configure Senzing

Next, we'll configure the Senzing engine to accept the `reference` data, in the same way as the `spark_quickstart` tutorial.

In [ ]:
grpc_channel = grpc.insecure_channel("localhost:8261")
sz_abstract_factory = SzAbstractFactoryGrpc(grpc_channel)

We'll check connectivity by getting the Senzing version:

In [ ]:
sz_product = sz_abstract_factory.create_product()
print(json.dumps(json.loads(sz_product.get_version()), indent=2))

In [ ]:
sz_configmanager = sz_abstract_factory.create_configmanager()
sz_diagnostic = sz_abstract_factory.create_diagnostic()
sz_engine = sz_abstract_factory.create_engine()

In [ ]:
config_id = sz_configmanager.get_default_config_id()
sz_config = sz_configmanager.create_config_from_config_id(config_id)

This time, we'll only use a single data source:

In [ ]:
try:
    sz_config.register_data_source('REFERENCE')

except (grpc.RpcError, szerror.SzError) as err:
    print(err)
    print('You only need to register the data source once')

In [ ]:
new_json_config = sz_config.export()
new_config_id = sz_configmanager.register_config(new_json_config, "Spark Streaming")
sz_configmanager.replace_default_config_id(config_id, new_config_id)

In [ ]:
sz_abstract_factory.reinitialize(new_config_id)

## Set up Spark streaming functions


We'll start a new Spark session, create a schema, and then set up a stream reader from Spark's [Structured Streaming](https://spark.apache.org/docs/latest/streaming/index.html) engine. In the next section, we'll use a stream writer to send the data from the Spark streaming dataframe to Senzing.

First, we'll create a Spark session:

In [ ]:
spark = SparkSession.builder \
    .appName("Senzing Streaming") \
    .master("local[*]") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/checkpoint") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

Providing a schema for our data makes sure that all the files have the correct information, and also speeds up the Spark stream reader.

In [ ]:
customers_schema = StructType([
    StructField("DATA_SOURCE", StringType(), True),
    StructField("RECORD_ID", StringType(), True),
    StructField("RECORD_TYPE", StringType(), True),
    StructField("PRIMARY_NAME_ORG", StringType(), True),
    StructField("SECONDARY_NAME_ORG", StringType(), True),
    StructField("PRIMARY_NAME_FULL", StringType(), True),
    StructField("NATIVE_NAME_FULL", StringType(), True),
    StructField("PRIMARY_NAME_LAST", StringType(), True),
    StructField("PRIMARY_NAME_FIRST", StringType(), True),
    StructField("PRIMARY_NAME_MIDDLE", StringType(), True),
    StructField("GENDER", StringType(), True),
    StructField("DATE_OF_BIRTH", StringType(), True),
    StructField("PASSPORT_NUMBER", StringType(), True),
    StructField("PASSPORT_COUNTRY", StringType(), True),
    StructField("DRIVERS_LICENSE_NUMBER", StringType(), True),
    StructField("DRIVERS_LICENSE_STATE", StringType(), True),
    StructField("SSN_NUMBER", StringType(), True),
    StructField("NATIONAL_ID_NUMBER", StringType(), True),
    StructField("NATIONAL_ID_COUNTRY", StringType(), True),
    StructField("ADDR_TYPE", StringType(), True),
    StructField("ADDR_FULL", StringType(), True),
    StructField("ADDR_LINE1", StringType(), True),
    StructField("ADDR_CITY", StringType(), True),
    StructField("ADDR_STATE", StringType(), True),
    StructField("ADDR_POSTAL_CODE", StringType(), True),
    StructField("ADDR_COUNTRY", StringType(), True),
    StructField("PHONE_TYPE", StringType(), True),
    StructField("PHONE_NUMBER", StringType(), True),
    StructField("EMAIL_ADDRESS", StringType(), True),
    StructField("DATE", StringType(), True),
    StructField("STATUS", StringType(), True),
    StructField("CATEGORY", StringType(), True),
    StructField("AMOUNT", StringType(), True)
])

The stream reader uses this schema to write to a streaming dataframe. For this example, it reads one file at a time to simulate streaming, but you can easily change this to your input stream.

In [ ]:
streaming_df = spark \
    .readStream \
    .schema(customers_schema) \
    .option("maxFilesPerTrigger", 1)  \
    .json('data/streaming')

## Add records to Senzing and to Spark dataframe

We'll use the `get_affected_entities` function from the `spark_quickstart` tutorial to track what entities have been changed or created in the Senzing repository:

In [ ]:
def get_affected_entities(info_string):
    # helper function to extract the entity id
    info = json.loads(info_string)
    return [entity['ENTITY_ID'] for entity in info['AFFECTED_ENTITIES']]

In [ ]:
affected_entities = set()

And we'll use the code from the `spark_quickstart` tutorial to create a function that will send a streaming batch to the Senzing engine:

In [ ]:
def process_streaming_batch(batch_df, batch_id):
    
    if batch_df.count() == 0:
        return

    print(f"Processing batch {batch_id} with {batch_df.count()} records")
    
    for row in batch_df.rdd.toLocalIterator():
        record = {k: v for k, v in row.asDict().items() if v is not None}
        
        info = sz_engine.add_record(
            record['DATA_SOURCE'],
            record['RECORD_ID'], 
            record,
            szengineflags.SzEngineFlags.SZ_WITH_INFO,
        )
        
        affected_entities.update(get_affected_entities(info))
        print(f"Added record {record['RECORD_ID']}")

Then, we'll stream the data from the Spark dataframe to Senzing using a Spark stream writer:

In [ ]:
streaming_query = streaming_df \
    .writeStream \
    .foreachBatch(process_streaming_batch) \
    .outputMode("append") \
    .trigger(processingTime='10 seconds') \
    .start()

While this is running, we can get a count of the number of records that have been added to the Senzing repository:

In [ ]:
result = json.loads(sz_engine.get_stats())
print(result['workload']['loadedRecords'])

We can view the `affected_entities` set to confirm that entities have been created:

In [ ]:
affected_entities

And, for each of the entities that have been updated, we can pull the details of that entity:

In [ ]:
for entity_id in affected_entities:
    result = json.loads(sz_engine.get_entity_by_entity_id(entity_id))
    print(f"Entity ID: {str(entity_id)}, Name: {result['RESOLVED_ENTITY']['ENTITY_NAME']}, Record Type: {result['RESOLVED_ENTITY']['FEATURES']['RECORD_TYPE'][0]['FEAT_DESC']}")

If you want to scale this up, you can use the `ThreadPoolExecutor` from the Python [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) module, as shown in this example: https://github.com/brianmacy/sz_incremental_withinfo-v3/blob/main/sz_incremental_withinfo.py

## Process REDO records

As in the `spark_quickstart` tutorial, we'll run the Senzing [redo process](https://senzing.zendesk.com/hc/en-us/articles/360007475133-Processing-REDO) to clean up the entities in the Senzing repository, updating the `affected_entities` set as we go.

In [ ]:
while True:
    redo_record = sz_engine.get_redo_record()
    if not redo_record:
        break
    info = sz_engine.process_redo_record(redo_record, flags=SzEngineFlags.SZ_WITH_INFO)
    affected_entities.update(get_affected_entities(info))
    print(info)

## Look up entities

Now that the entities have been resolved, we can search the Senzing repository for anyone we are interested in:

In [ ]:
search_query = {
    "name_full": "Wang Jie",
}
search_result = sz_engine.search_by_attributes(json.dumps(search_query))
print(json.dumps(json.loads(search_result), indent=2))

You can take a look at the `spark_quickstart` tutorial for details on how to extract data from the Senzing repository.

In [ ]:
spark.stop()

## Next steps

If you’re interested in exploring Senzing further, check out the following links:

- [Senzing + Docker quickstart](https://senzing.com/docs/quickstart/quickstart_docker/)

- [Senzing Learning Portal](https://senzing.com/senzing-learning-portal-signup)

- [Senzing SDK Documentation](https://senzing.com/docs/)

- [Entity Centric Learning](https://senzing.com/entity-centric-learning-explained/)

- [CORD: Collections Of Relatable Data](https://senzing.com/senzing-ready-data-collections-cord)

- [Senzing GitHub public repos](https://github.com/senzing-garage)

- ["Graph Power Hour!" podcast](https://senzing.com/graph-power-hour)